In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

import math

import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import KFold

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [66]:

"""
Compute the entropy of input series
"""
def findFlakyEntropy(input):
    entropy = 0

    unique, counts = np.unique(input, return_counts=True)
    input_dict = dict(zip(unique, counts))
    categories = input_dict.keys()
    base = len(categories)

    for category in categories:
        p_category = input_dict[category]/input.size
        entropy = entropy - p_category * math.log(p_category, base)
    
    return entropy

"""
Compute the mutual info between input series and output ndarray
"""
def findFlakyMutualInformation(input, output):
    mutualInfo = 0

    """
    categories = input.unique()
    input_dict = input.value_counts().to_dict()
    base = len(categories)
    """

    unique, counts = np.unique(input, return_counts=True)
    input_dict = dict(zip(unique, counts))
    categories = input_dict.keys()
    base = len(categories)

    unique, counts = np.unique(output, return_counts=True)
    output_dict = dict(zip(unique, counts))

    for category in categories:
        if category not in output_dict:
            output_dict[category] = 0

    for x_category in categories:
        for y_category in categories:
            index = 0 
            xy_occurrence = 0
            for i, v in enumerate(input):
                if v == x_category and output[i] == y_category:
                    xy_occurrence = xy_occurrence + 1
                index = index +1
            
            p_xy = xy_occurrence/input.size
            p_x = input_dict[x_category]/input.size
            p_y = output_dict[y_category]/input.size

            if p_xy > 0:
                mutualInfo = mutualInfo + p_xy * math.log(p_xy/(p_x*p_y), base)

    return mutualInfo

"""
Compute the flaky detection capacity from input series and output ndarray
Based on intrusion detection capacity
"""
def findFlakyDetectionCapacity(input, output):
    mutualInfo = findFlakyMutualInformation(input, output)
    entropy = findFlakyEntropy(input)

    return mutualInfo/entropy


def samplingSMOTEandTL(vector_val, vector_cat):
    smote = SMOTE(k_neighbors=3)
    X_smote, y_smote = smote.fit_resample(vector_val, vector_cat)

    tl = TomekLinks()
    X_tl, y_tl = tl.fit_resample(X_smote, y_smote)

    return X_tl, y_tl

1. Load code2vec
2. Run LDA
3. Split, train and test on 10 sub test data set
4. Store F1 score and fdc
5. Shuffle and repeat 3 & 4
6. For every combination of sub data set, record the consistency and discriminency
7. Compute D and C

In [5]:
dataCSV = pd.read_csv('extracted-all-projects.csv')
vectorCSV = pd.read_csv('vector.csv')

raw = dataCSV['raw']
dataCSV['id']=range(1, len(dataCSV) + 1)
vectorCSV['id']=range(1, len(vectorCSV) + 1)
switchedCSV = vectorCSV[['id','name','feature_0','feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15','feature_16','feature_17','feature_18','feature_19','feature_20','feature_21','feature_22','feature_23','feature_24','feature_25','feature_26','feature_27','feature_28','feature_29','feature_30','feature_31','feature_32','feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_39','feature_40','feature_41','feature_42','feature_43','feature_44','feature_45','feature_46','feature_47','feature_48','feature_49','feature_50','feature_51','feature_52','feature_53','feature_54','feature_55','feature_56','feature_57','feature_58','feature_59','feature_60','feature_61','feature_62','feature_63','feature_64','feature_65','feature_66','feature_67','feature_68','feature_69','feature_70','feature_71','feature_72','feature_73','feature_74','feature_75','feature_76','feature_77','feature_78','feature_79','feature_80','feature_81','feature_82','feature_83','feature_84','feature_85','feature_86','feature_87','feature_88','feature_89','feature_90','feature_91','feature_92','feature_93','feature_94','feature_95','feature_96','feature_97','feature_98','feature_99','feature_100','feature_101','feature_102','feature_103','feature_104','feature_105','feature_106','feature_107','feature_108','feature_109','feature_110','feature_111','feature_112','feature_113','feature_114','feature_115','feature_116','feature_117','feature_118','feature_119','feature_120','feature_121','feature_122','feature_123','feature_124','feature_125','feature_126','feature_127','feature_128','feature_129','feature_130','feature_131','feature_132','feature_133','feature_134','feature_135','feature_136','feature_137','feature_138','feature_139','feature_140','feature_141','feature_142','feature_143','feature_144','feature_145','feature_146','feature_147','feature_148','feature_149','feature_150','feature_151','feature_152','feature_153','feature_154','feature_155','feature_156','feature_157','feature_158','feature_159','feature_160','feature_161','feature_162','feature_163','feature_164','feature_165','feature_166','feature_167','feature_168','feature_169','feature_170','feature_171','feature_172','feature_173','feature_174','feature_175','feature_176','feature_177','feature_178','feature_179','feature_180','feature_181','feature_182','feature_183','feature_184','feature_185','feature_186','feature_187','feature_188','feature_189','feature_190','feature_191','feature_192','feature_193','feature_194','feature_195','feature_196','feature_197','feature_198','feature_199','feature_200','feature_201','feature_202','feature_203','feature_204','feature_205','feature_206','feature_207','feature_208','feature_209','feature_210','feature_211','feature_212','feature_213','feature_214','feature_215','feature_216','feature_217','feature_218','feature_219','feature_220','feature_221','feature_222','feature_223','feature_224','feature_225','feature_226','feature_227','feature_228','feature_229','feature_230','feature_231','feature_232','feature_233','feature_234','feature_235','feature_236','feature_237','feature_238','feature_239','feature_240','feature_241','feature_242','feature_243','feature_244','feature_245','feature_246','feature_247','feature_248','feature_249','feature_250','feature_251','feature_252','feature_253','feature_254','feature_255','feature_256','feature_257','feature_258','feature_259','feature_260','feature_261','feature_262','feature_263','feature_264','feature_265','feature_266','feature_267','feature_268','feature_269','feature_270','feature_271','feature_272','feature_273','feature_274','feature_275','feature_276','feature_277','feature_278','feature_279','feature_280','feature_281','feature_282','feature_283','feature_284','feature_285','feature_286','feature_287','feature_288','feature_289','feature_290','feature_291','feature_292','feature_293','feature_294','feature_295','feature_296','feature_297','feature_298','feature_299','feature_300','feature_301','feature_302','feature_303','feature_304','feature_305','feature_306','feature_307','feature_308','feature_309','feature_310','feature_311','feature_312','feature_313','feature_314','feature_315','feature_316','feature_317','feature_318','feature_319','feature_320','feature_321','feature_322','feature_323','feature_324','feature_325','feature_326','feature_327','feature_328','feature_329','feature_330','feature_331','feature_332','feature_333','feature_334','feature_335','feature_336','feature_337','feature_338','feature_339','feature_340','feature_341','feature_342','feature_343','feature_344','feature_345','feature_346','feature_347','feature_348','feature_349','feature_350','feature_351','feature_352','feature_353','feature_354','feature_355','feature_356','feature_357','feature_358','feature_359','feature_360','feature_361','feature_362','feature_363','feature_364','feature_365','feature_366','feature_367','feature_368','feature_369','feature_370','feature_371','feature_372','feature_373','feature_374','feature_375','feature_376','feature_377','feature_378','feature_379','feature_380','feature_381','feature_382','feature_383']]
switchedCSV['category'] = dataCSV['category']
switchedCSV['category'].replace(np.nan, 'NonFlaky', inplace=True)
switchedCSV = switchedCSV[(switchedCSV['category'] == 'ID') | (switchedCSV['category'] == 'UD') |(switchedCSV['category'] == 'OD') | (switchedCSV['category'] == 'OD-Vic') | (switchedCSV['category'] == 'OD-Brit') | (switchedCSV['category'] == 'NOD') | (switchedCSV['category'] == 'NDOD') | (switchedCSV['category'] == 'NODI')]
features = ['feature_0','feature_1','feature_2','feature_3','feature_4','feature_5','feature_6','feature_7','feature_8','feature_9','feature_10','feature_11','feature_12','feature_13','feature_14','feature_15','feature_16','feature_17','feature_18','feature_19','feature_20','feature_21','feature_22','feature_23','feature_24','feature_25','feature_26','feature_27','feature_28','feature_29','feature_30','feature_31','feature_32','feature_33','feature_34','feature_35','feature_36','feature_37','feature_38','feature_39','feature_40','feature_41','feature_42','feature_43','feature_44','feature_45','feature_46','feature_47','feature_48','feature_49','feature_50','feature_51','feature_52','feature_53','feature_54','feature_55','feature_56','feature_57','feature_58','feature_59','feature_60','feature_61','feature_62','feature_63','feature_64','feature_65','feature_66','feature_67','feature_68','feature_69','feature_70','feature_71','feature_72','feature_73','feature_74','feature_75','feature_76','feature_77','feature_78','feature_79','feature_80','feature_81','feature_82','feature_83','feature_84','feature_85','feature_86','feature_87','feature_88','feature_89','feature_90','feature_91','feature_92','feature_93','feature_94','feature_95','feature_96','feature_97','feature_98','feature_99','feature_100','feature_101','feature_102','feature_103','feature_104','feature_105','feature_106','feature_107','feature_108','feature_109','feature_110','feature_111','feature_112','feature_113','feature_114','feature_115','feature_116','feature_117','feature_118','feature_119','feature_120','feature_121','feature_122','feature_123','feature_124','feature_125','feature_126','feature_127','feature_128','feature_129','feature_130','feature_131','feature_132','feature_133','feature_134','feature_135','feature_136','feature_137','feature_138','feature_139','feature_140','feature_141','feature_142','feature_143','feature_144','feature_145','feature_146','feature_147','feature_148','feature_149','feature_150','feature_151','feature_152','feature_153','feature_154','feature_155','feature_156','feature_157','feature_158','feature_159','feature_160','feature_161','feature_162','feature_163','feature_164','feature_165','feature_166','feature_167','feature_168','feature_169','feature_170','feature_171','feature_172','feature_173','feature_174','feature_175','feature_176','feature_177','feature_178','feature_179','feature_180','feature_181','feature_182','feature_183','feature_184','feature_185','feature_186','feature_187','feature_188','feature_189','feature_190','feature_191','feature_192','feature_193','feature_194','feature_195','feature_196','feature_197','feature_198','feature_199','feature_200','feature_201','feature_202','feature_203','feature_204','feature_205','feature_206','feature_207','feature_208','feature_209','feature_210','feature_211','feature_212','feature_213','feature_214','feature_215','feature_216','feature_217','feature_218','feature_219','feature_220','feature_221','feature_222','feature_223','feature_224','feature_225','feature_226','feature_227','feature_228','feature_229','feature_230','feature_231','feature_232','feature_233','feature_234','feature_235','feature_236','feature_237','feature_238','feature_239','feature_240','feature_241','feature_242','feature_243','feature_244','feature_245','feature_246','feature_247','feature_248','feature_249','feature_250','feature_251','feature_252','feature_253','feature_254','feature_255','feature_256','feature_257','feature_258','feature_259','feature_260','feature_261','feature_262','feature_263','feature_264','feature_265','feature_266','feature_267','feature_268','feature_269','feature_270','feature_271','feature_272','feature_273','feature_274','feature_275','feature_276','feature_277','feature_278','feature_279','feature_280','feature_281','feature_282','feature_283','feature_284','feature_285','feature_286','feature_287','feature_288','feature_289','feature_290','feature_291','feature_292','feature_293','feature_294','feature_295','feature_296','feature_297','feature_298','feature_299','feature_300','feature_301','feature_302','feature_303','feature_304','feature_305','feature_306','feature_307','feature_308','feature_309','feature_310','feature_311','feature_312','feature_313','feature_314','feature_315','feature_316','feature_317','feature_318','feature_319','feature_320','feature_321','feature_322','feature_323','feature_324','feature_325','feature_326','feature_327','feature_328','feature_329','feature_330','feature_331','feature_332','feature_333','feature_334','feature_335','feature_336','feature_337','feature_338','feature_339','feature_340','feature_341','feature_342','feature_343','feature_344','feature_345','feature_346','feature_347','feature_348','feature_349','feature_350','feature_351','feature_352','feature_353','feature_354','feature_355','feature_356','feature_357','feature_358','feature_359','feature_360','feature_361','feature_362','feature_363','feature_364','feature_365','feature_366','feature_367','feature_368','feature_369','feature_370','feature_371','feature_372','feature_373','feature_374','feature_375','feature_376','feature_377','feature_378','feature_379','feature_380','feature_381','feature_382','feature_383']
vector_val = switchedCSV[features]
vector_cat = switchedCSV['category']
vector_cat_val = vector_cat.values

lda = LinearDiscriminantAnalysis(n_components=3)
lda_val = lda.fit_transform(vector_val, vector_cat)

/var/folders/s8/vyj613fs57b4g86vrn1xc5t80000gn/T/ipykernel_30888/1000679553.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  switchedCSV['category'] = dataCSV['category']
/usr/local/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [59]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

svm_classifier = svm.SVC(kernel='rbf', random_state=0)

config = {
    'max_features': 0.5247945548949592, 
    'max_samples': 0.6302324164385819, 
    'min_impurity_decrease': 0.0015368114684006229, 
    'min_samples_leaf': 41, 
    'min_samples_split': 40, 
    'n_estimators': 115
    }

rfc = RandomForestClassifier(
    max_features=config['max_features'],
    max_samples=config['max_samples'],
    min_impurity_decrease=config['min_impurity_decrease'],
    min_samples_leaf=config['min_samples_leaf'],
    min_samples_split=config['min_samples_split'],
    n_estimators=config['n_estimators'],
    random_state=0)


config = {
    'learning_rate': 0.5433990284529154, 
    'max_depth': 17, 
    'min_samples_leaf': 88, 
    'min_samples_split': 305, 
    'n_estimators': 187
    }

gbdt = GradientBoostingClassifier(
    learning_rate=config['learning_rate'],
    max_depth=config['max_depth'],
    min_samples_leaf=config['min_samples_leaf'],
    min_samples_split=config['min_samples_split'],
    n_estimators=config['n_estimators'],
    random_state=0)

In [69]:
def runKFold(classifier, fold, runs):
    sum_f1s = 0
    sum_fdc = 0
    result = {"f1s": [], "fdc": []}

    for i in range(runs):
            
        kf = KFold(n_splits=fold, shuffle=True, random_state=i)   
        for train_index, test_index in kf.split(lda_val):
            X_train, X_test = lda_val[train_index], lda_val[test_index]
            y_train, y_test = vector_cat_val[train_index], vector_cat_val[test_index]

            classifier.fit(X_train,y_train)
            y_pred = classifier.predict(X_test)
            
            sum_f1s += f1_score(y_test, y_pred, average='weighted')
            sum_fdc += findFlakyDetectionCapacity(y_test, y_pred)
            result["f1s"].append(f1_score(y_test, y_pred, average='weighted'))
            result["fdc"].append(findFlakyDetectionCapacity(y_test, y_pred))
            
    C = computeC(result["fdc"], result["f1s"])
    print(C)
    D = computeD(result["fdc"], result["f1s"])
    print(D)
    

def runBalancedKFold(classifier, fold, runs):
    sum_f1s = 0
    sum_fdc = 0
    result = {"f1s": [], "fdc": []}

    # explore the effect of size of overall calculation test size.
    # might also try different train and test ratio
    for i in range(runs):
            
        kf = KFold(n_splits=fold, shuffle=True, random_state=i)   
        for train_index, test_index in kf.split(lda_val):
            X_train, X_test = lda_val[train_index], lda_val[test_index]
            y_train, y_test = vector_cat_val[train_index], vector_cat_val[test_index]

            X_sampled, y_sampled = samplingSMOTEandTL(X_train, y_train)

            classifier.fit(X_sampled,y_sampled)
            y_pred = classifier.predict(X_test)
            
            sum_f1s += f1_score(y_test, y_pred, average='weighted')
            sum_fdc += findFlakyDetectionCapacity(y_test, y_pred)
            result["f1s"].append(f1_score(y_test, y_pred, average='weighted'))
            result["fdc"].append(findFlakyDetectionCapacity(y_test, y_pred))
            
    C = computeC(result["fdc"], result["f1s"])
    print(C)
    D = computeD(result["fdc"], result["f1s"])
    print(D)

2 list
We want to count whether fdc(a)>fdc(b) gives f1s(a)>f1s(b)

In [63]:
def computeC(listFDC, listF1s):
    pairs = list(itertools.combinations(range(len(listFDC)),2))
    consistent = 0
    inconsistent = 0

    for pair in pairs:
        if listFDC[pair[0]] > listFDC[pair[1]] and listF1s[pair[0]] > listF1s[pair[1]]:
            consistent += 1
        elif listFDC[pair[0]] < listFDC[pair[1]] and listF1s[pair[0]] < listF1s[pair[1]]:
            consistent += 1
        else:
            inconsistent += 1
    C = consistent / (consistent + inconsistent)
    return C

def computeD(listFDC, listF1s):
    pairs = list(itertools.combinations(range(len(listFDC)),2))
    discriminant = 0
    indiscriminant = 0

    listFDC = [ round(elem, 2) for elem in listFDC ]
    listF1s = [ round(elem, 2) for elem in listF1s ]

    for pair in pairs:
        if listFDC[pair[0]] != listFDC[pair[1]] and listF1s[pair[0]] == listF1s[pair[1]]:
            discriminant += 1
        elif listFDC[pair[0]] == listFDC[pair[1]] and listF1s[pair[0]] != listF1s[pair[1]]:
            indiscriminant += 1
    if indiscriminant == 0:
        indiscriminant = 1
    D = discriminant/indiscriminant
    return D


In [70]:
runKFold(knn, 5, 5)
runKFold(svm_classifier, 5, 5)
runKFold(rfc, 5, 5)
runKFold(gbdt, 5, 5)

0.8666666666666667
1.9285714285714286
0.8566666666666667
1.8571428571428572
0.7733333333333333
1.3181818181818181
0.8233333333333334
2.0


In [76]:
runKFold(knn, 5, 10)
runKFold(svm_classifier, 5, 10)
runKFold(rfc, 5, 10)
runKFold(gbdt, 5, 10)

0.8628571428571429
2.24
0.8571428571428571
1.9333333333333333
0.7861224489795918
1.7794117647058822
0.8122448979591836
1.9295774647887325


In [77]:
runKFold(knn, 5, 20)
runKFold(svm_classifier, 5, 20)
runKFold(rfc, 5, 20)
runKFold(gbdt, 5, 20)

0.8276767676767677
2.029739776951673
0.8446464646464646
1.8847583643122676
0.7933333333333333
1.7207547169811321
0.807070707070707
2.11231884057971


In [78]:
runKFold(knn, 5, 50)
runKFold(svm_classifier, 5, 50)
runKFold(rfc, 5, 50)
runKFold(gbdt, 5, 50)

0.845429718875502
1.9169632265717675
0.8468433734939759
1.889022195560888
0.7974939759036145
1.5561766349916155
0.8259277108433735
1.9489795918367347


In [75]:
runBalancedKFold(knn, 5, 5)
runBalancedKFold(svm_classifier, 5, 5)
runBalancedKFold(rfc, 5, 5)
runBalancedKFold(gbdt, 5, 5)

0.7566666666666667
1.25
0.8033333333333333
2.066666666666667
0.7766666666666666
1.65
0.7966666666666666
2.125


In [74]:
runBalancedKFold(knn, 5, 10)
runBalancedKFold(svm_classifier, 5, 10)
runBalancedKFold(rfc, 5, 10)
runBalancedKFold(gbdt, 5, 10)

0.7959183673469388
2.7450980392156863
0.773061224489796
1.7922077922077921
0.7534693877551021
2.0416666666666665
0.8367346938775511
2.298507462686567


In [72]:
runBalancedKFold(knn, 5, 20)
runBalancedKFold(svm_classifier, 5, 20)
runBalancedKFold(rfc, 5, 20)
runBalancedKFold(gbdt, 5, 20)

0.8234343434343434
1.8856209150326797
0.7882828282828283
2.1241610738255035
0.7864646464646464
1.911854103343465
0.7808080808080808
1.8227848101265822


In [73]:
runBalancedKFold(knn, 5, 50)
runBalancedKFold(svm_classifier, 5, 50)
runBalancedKFold(rfc, 5, 50)
runBalancedKFold(gbdt, 5, 50)

0.8158072289156626
1.9714774671990873
0.804433734939759
1.7717948717948717
0.7980722891566265
1.8634666666666666
0.8161285140562249
1.8624052004333695


In [ ]:
@Test public void testDynamicDeleteException() {
    SqlSession sqlSession=MybatisHelper.getSqlSession();
    try{
        CountryMapper mapper=sqlSession.getMapper(CountryMapper.class);
        Assert.assertEquals(1,mapper.deleteByPrimaryKey(100));
    }
    finally{
        sqlSession.rollback();sqlSession.close();
    }
}
